In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama3-70b-8192"
)

In [2]:
response = llm.invoke("The first person to won fifa worldcup..")
print(response.content)

The first person to win the FIFA World Cup was Uruguayan captain José Nasazzi, who led his team to victory in the inaugural FIFA World Cup held in 1930 in Uruguay.


In [3]:
from langchain_community.document_loaders import WebBaseLoader     #To scrap data from any website

loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-itc/job/R-49963")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Software Engineer -II, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu




In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer -II, ITC',
  'experience': '2+ years of hands-on industry software development experience',
  'skills': ['front-end frameworks like React, Angular, or Vue.js',
   'cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
   'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
   'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
   'Strong understanding of architectural design patterns and computer-science fundamentals',
   'Experience with implementing and integrating AI, Machine Learning and related data solutions'],
  'description': 'We’re looking for a Software Engineer to solve complex software engineering problems supporting Nike’s pursuit of delivering state of the art tools to our Consumer Product & Innovation community.'}]

In [6]:
type(json_res)

list

In [7]:
import pandas as pd

df= pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\lenovo\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|█|


In [13]:
links = collection.query(query_texts=['Expertise in machine learning'], n_results=2).get('metadatas', [])  #Simple collection query
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [16]:
job = json_res[0]

In [17]:
job

{'role': 'Software Engineer -II, ITC',
 'experience': '2+ years of hands-on industry software development experience',
 'skills': ['front-end frameworks like React, Angular, or Vue.js',
  'cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
  'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
  'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
  'Strong understanding of architectural design patterns and computer-science fundamentals',
  'Experience with implementing and integrating AI, Machine Learning and related data solutions'],
 'description': 'We’re looking for a Software Engineer to solve complex software engineering problems supporting Nike’s pursuit of delivering state of the art tools to our Consumer Product & Innovation community.'}

In [18]:
job['skills']

['front-end frameworks like React, Angular, or Vue.js',
 'cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
 'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
 'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
 'Strong understanding of architectural design patterns and computer-science fundamentals',
 'Experience with implementing and integrating AI, Machine Learning and related data solutions']

In [19]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}]]

In [21]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Anami, a final-year undergraduate student actively seeking opportunities in industry. 
    You have strong interest in this role and want to demonstrate your alignment with the requirements mentioned above. 
    Write a professional cold email to the company referring to the job description, expressing your interest, 
    summarizing why you’re a good fit (based on their needs), and conveying your enthusiasm to contribute.
    
    Also, include the most relevant ones from the following links to highlight your previous projects or work that aligns with their role: {link_list}
    
    Keep the tone respectful, confident, and professional — but make sure it clearly sounds like a student reaching out for an opportunity.
    
    Do not provide a preamble.
    
    ### EMAIL (NO PREAMBLE):
    
    """
    )
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineer - II, ITC Role at Nike

Dear Hiring Manager,

I am excited to express my interest in the Software Engineer - II, ITC role at Nike, as described in the job description. With a strong passion for software development and a keen interest in AI, Machine Learning, and cloud architecture, I believe I would be a great fit for this position.

Throughout my undergraduate studies, I have developed a solid foundation in computer-science fundamentals and architectural design patterns. I have hands-on experience with front-end frameworks like Vue.js, having built a portfolio of projects that demonstrate my skills in this area (https://example.com/vue-portfolio). Additionally, I have explored modern DevOps practices and infrastructure as code, which I believe would be an asset in this role.

I am particularly drawn to this role at Nike because of the opportunity to work on complex software engineering problems that support the company's pursuit of deliverin